In [14]:
import torch
import numpy as np
import cudf
from tqdm import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
df = cudf.read_csv("../dataset/imet/train.csv")
df.shape

(142119, 2)

In [3]:
df.head()

,id,attribute_ids
0,000040d66f14ced4cdd18cd95d91800f,448 2429 782
1,0000ef13e37ef70412166725ec034a8a,2997 3231 2730 3294 3099 2017 784
2,0001eeb4a06e8daa7c6951bcd124c3c7,2436 1715 23
3,000226398d224de78b191e6db45fd94e,2997 3433 448 782
4,00029c3b0171158d63b1bbf803a7d750,3465 3322 3170 1553 781


In [4]:
N_CLASSES = 3474

In [5]:
target = torch.zeros(len(df), N_CLASSES)

In [6]:
target.shape

torch.Size([142119, 3474])

In [7]:
df.iloc[0, 1].split()

['448', '2429', '782']

In [8]:
for i in tqdm(range(len(df))):
    for attribute_id in df.iloc[i, 1].split():
        target[i, int(attribute_id)] = 1

100%|██████████| 142119/142119 [03:59<00:00, 593.37it/s]


In [9]:
target.sum(0)

tensor([ 9.,  3., 10.,  ...,  3., 30., 34.])

In [10]:
target = target.to('cpu').detach().numpy()

In [11]:
target

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
X = df["id"].to_array()

In [13]:
X

array(['000040d66f14ced4cdd18cd95d91800f',
       '0000ef13e37ef70412166725ec034a8a',
       '0001eeb4a06e8daa7c6951bcd124c3c7', ...,
       'fffdde4325edb076dccea37978c1378a',
       'fffe25dccc93a5e9e12f26d494dae26f',
       'fffe66c4a175c96065551cebacb7636f'], dtype=object)

In [15]:
mskf = MultilabelStratifiedKFold(
    n_splits=5, shuffle=True, random_state=1031
)

/home/karunru/.pyenv/versions/miniconda3-4.7.12/envs/rapids-0.19/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=1031 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [17]:
for fold, (train_idx, val_idx) in enumerate(tqdm(mskf.split(X, target))):
    np.save(f"train_idx_fold{fold}.npy", train_idx)
    np.save(f"val_idx_fold{fold}.npy", val_idx)
    print(f"fold {fold}")

5it [15:30, 186.11s/it]

fold 0
fold 1
fold 2
fold 3
fold 4
